## Cleaning up and transforming data in the raw excel sheet


In [19]:
import pandas as pd

data = pd.read_excel("rawData.xlsx")

### 1. Clean up Author Names

Replaced letters with accents with it's anglaphied pair, removed occurences of the character ~, and removed curly brackets around a word({ }).


In [20]:
import re

def remove_tex_accents(text):
    patterns = [
        (r"{\\'\s*(\w)}", r"\1"),     # \'e -> e
        (r"{\\`\s*(\w)}", r"\1"),     # \`e -> e
        (r"{\\\^\s*(\w)}", r"\1"),     # \^e -> e
        (r"{\\\"\s*(\w)}", r"\1"),    # \"e -> e
        (r"{\\~\s*(\w)}", r"\1"),     # \~e -> e
        (r"{\\=\s*(\w)}", r"\1"),     # \=e -> e
        (r"{\\u\s*(\w)}", r"\1"),     # \ue -> e
        (r"{\\v\s*(\w)}", r"\1"),     # \ve -> e
        (r"{\\c\s*(\w)}", r"\1"),     # \ce -> e
        (r"{\\d\s*(\w)}", r"\1"),     # \de -> e
        (r"{\\H\s*(\w)}", r"\1"),     # \He -> e
        (r"{\\t\s*(\w)}", r"\1"),     # \te -> e
        (r"{\\b\s*(\w)}", r"\1"),     # \be -> e
        (r"{\\k\s*(\w)}", r"\1"),     # \ke -> e
        (r"{\\r\s*(\w)}", r"\1"),     # \re -> e
        (r"{\\c\s*(\w)}",r"\1"),     # {\c c} -> c
        (r"{\\H\s*(\w)}",r"\1"),     # {\H c} -> c
        (r"{\\(ae)}",r"\1"),     #{\ae} -> ae
        (r"{(\S*)}",r"\1"),      #{someLetters} -> someLetters 
        (r"(~)",r"")             # ~ -> 
    ]
#{\^o}
    for pattern, replacement in patterns:
        text = re.sub(pattern, replacement, text)

    return text

data["Old Authors Data"] = data["Authors"]
data["Authors"] = data["Authors"].map(lambda d: remove_tex_accents(d))


## 2. Clean up the title

Replaced {{ }} or {{{ }}} surrounding words, with just the inner words, and replaced occurences of -- or --- with just a -


In [21]:
def cleanUpTitle(text):
    
    patterns = [
        (r'{{|}}', ''),
        (r'---|--', '-')
    ]

    for pattern, replacement in patterns:
        text = re.sub(pattern, replacement, text)

    return text

data["Old Title"] = data["Title"]
data["Title"] = data["Title"].map(lambda d: cleanUpTitle(str(d)))

In [22]:
""" for index, row in data.iterrows():
    print(row['Title'])
    print(row['Old Title'])
    print(index)"""
#262 ``How Local Is Local?''---A Review of Practical and Conceptual Issues in the Genetics of Restoration
#362 is nan

" for index, row in data.iterrows():\n    print(row['Title'])\n    print(row['Old Title'])\n    print(index)"

## 3. Clean up Journal Title

Replaced '\\&' to '&'

I still have to replace some of na values.


In [23]:
#
def cleanUpJournal(text):
    patterns = [
        (r'\\&', r"&")
    ]
    for pattern, replacement in patterns:
        text = re.sub(pattern, replacement, text)
    return text


data["Old Journal"] = data["Journal"]
data["Journal"] = data["Journal"].map(lambda d: cleanUpJournal(str(d)))

## 4. Generate the Journal Abbreviations

I looked at all the unique journal names and made a dictionary with those names as the key, and then automatically use that dictionary to generate a Jounral Abbrevation column.


In [24]:
#pd.DataFrame({"Journal":(data["Journal"]).unique()})
data["Journal"].unique()

array(['Annals of the New York Academy of Sciences',
       'Annual Review of Ecology and Systematics',
       'Annual Review of Ecology, Evolution, and Systematics',
       'Annual Review of Genetics', 'Behavioral Ecology and Sociobiology',
       'BioScience', 'Bulletin of Mathematical Biology',
       'Conservation Biology', 'Discrete & Continuous Dynamical Systems',
       'Ecography', 'Ecology and Evolution', 'Ecology Letters',
       'Evolution: International Journal of Organic Evolution',
       'Evolution Letters', 'Evolutionary Applications',
       'Evolutionary Ecology', 'Evolutionary Ecology Research',
       'Frontiers in Ecology and Evolution', 'Frontiers in Plant Science',
       'Functional Ecology', 'Genetica', 'Genetical Research', 'Genetics',
       'Genetics Research', 'Genome', 'Heredity', 'Interface Focus',
       'International Journal of Ecology', 'Journal of Arid Land',
       'Journal of Ecology', 'Journal of Evolutionary Biology',
       'Journal of Heredity'

In [25]:
# Define the journal names and their abbreviations
journals_abbreviations = {
    'Annals of the New York Academy of Sciences': 'Ann. N. Y. Acad. Sci.',
    'Annual Review of Ecology and Systematics': 'Annu. Rev. Ecol. Syst.',
    'Annual Review of Ecology, Evolution, and Systematics': 'Annu. Rev. Ecol. Evol. Syst.',
    'Annual Review of Genetics': 'Annu. Rev. Genet.',
    'Behavioral Ecology and Sociobiology': 'Behav. Ecol. Sociobiol.',
    'BioScience': 'BioScience',
    'Bulletin of Mathematical Biology': 'Bull. Math. Biol.',
    'Conservation Biology': 'Conserv. Biol.',
    'Discrete & Continuous Dynamical Systems': 'Discrete Contin. Dyn. Syst.',## should double check
    'Ecography': 'Ecography',
    'Ecology and Evolution': 'Ecol. Evol.',
    'Ecology Letters': 'Ecol. Lett.',
    'Evolution: International Journal of Organic Evolution': 'Evolution',
    'Evolution Letters': 'Evol. Lett.',
    'Evolutionary Applications': 'Evol. Appl.',
    'Evolutionary Ecology': 'Evol. Ecol.',
    'Evolutionary Ecology Research': 'Evol. Ecol. Res.',
    'Frontiers in Ecology and Evolution': 'Front. Ecol. Evol.',
    'Frontiers in Plant Science': 'Front. Plant Sci.',
    'Functional Ecology': 'Funct. Ecol.',
    'Genetica': 'Genetica',
    'Genetical Research': 'Genet. Res. (Camb.)',### does this journal even exist
    'Genetics': 'Genetics',
    'Genetics Research': 'Genet. Res. (Camb.)',
    'Genome': 'Genome',
    'Heredity': 'Heredity (Hered.)', ## acording to wiki, but the brackets at the end looks off
    'Interface Focus': 'Interface Focus',
    'International Journal of Ecology': 'Int. J. Ecol.',   ## couldn't verify this, but should be valid iso4 format
    'Journal of Arid Land': 'J. Arid Land',  ## couldn't verify this, but should be valid iso4 format
    'Journal of Ecology': 'J. Ecol.',
    'Journal of Evolutionary Biology': 'J. Evol. Biol.',
    'Journal of Heredity': 'J. Hered.',
    'Journal of Mathematical Analysis and Applications': 'J. Math. Anal. Appl.',
    'Journal of Mathematical Biology': 'J. Math. Biol.',
    'Journal of The Royal Society Interface': 'J. R. Soc. Interface',
    'Journal of Theoretical Biology': 'J. Theor. Biol.',
    'Methods in Ecology and Evolution': 'Methods Ecol. Evol.',
    'Molecular Biology and Evolution': 'Mol. Biol. Evol.',
    'Molecular Ecology': 'Mol. Ecol.',
    'Nature': 'Nature',
    'Nature Climate Change': 'Nat. Clim. Chang.',
    'Nature Communications': 'Nat. Commun.',
    'Nature Ecology & Evolution': 'Nat. Ecol. Evol.',
    'Nature Reviews Genetics': 'Nat. Rev. Genet.',
    'New Phytologist': 'New Phytol.',
    'Nonlinearity': 'Nonlinearity',
    'Oikos': 'Oikos',
    'Philosophical Transactions of the Royal Society B: Biological Sciences': 'Phil. Trans. R. Soc. B',
    'Philosophical Transactions of the Royal Society of London. Series B, Biological Sciences': 'Philos. Trans. R. Soc. Lond. B Biol. Sci.', ## need to check if this is the same as last time, seems to be so according to https://www.jstor.org/journal/philtranroyasoc2
    'Physical Review Research': 'Phys. Rev. Res.',
    'PLOS Biology': 'PLOS Biol.',
    'PLOS Computational Biology': 'PLOS Comput. Biol.',
    'PLOS Genetics': 'PLoS Genet.', ## not sure if the o should be capitalized or not, for the PLOS above, wikipedia has the O capitalized, while for this one, the o is lower cased
    'PLOS ONE': 'PLOS ONE',
    'Population Ecology': 'Popul. Ecol.', ## flagging this
    'Proceedings of the National Academy of Sciences': 'Proc. Natl. Acad. Sci.',
    'Proceedings of the National Academy of Sciences of the United States of America': 'Proc. Natl. Acad. Sci. U.S.A.',
    'Proceedings of the Royal Society B: Biological Sciences': 'Proc. R. Soc. B',
    'Proceedings of the Royal Society of London. Series B: Biological Sciences': 'Proc. R. Soc. Lond. B Biol. Sci.',## cant verify this
    'Proceedings: Biological Sciences': 'Proc. Biol. Sci.', ## seems to be the newer version of Proceedings of the Royal Society B: Biological Sciences, as said https://www.jstor.org/journal/procbiolscie
    'Restoration Ecology': 'Restor. Ecol.',
    'Science': 'Science',
    'Selection': 'Selection', ## cant verify this one, seems to be discontinued https://akjournals.com/view/journals/076/076-overview.xml
    'Spatial Statistics': 'Spat. Stat.',
    'The American Naturalist': 'Am. Nat.',
    'Theoretical Ecology': 'Theor. Ecol.',
    'Theoretical Population Biology': 'Theor. Popul. Biol.',
    'Trends in Ecology & Evolution': 'Trends Ecol. Evol.',
    'Trends in Genetics': 'Trends Genet.',
    'Trends in Parasitology': 'Trends Parasitol. (Parasitol. Today)' # flagging this, this journal used to be called Parasitology Today
}


data["Journal ISO Abbreviation"] = data["Journal"].map(journals_abbreviations)


### 5. Generate Citation Keys

Logic for generating Citation Key name (will need author names and date):

- if one author abc then paper name = "abc " + date
- if two author abc and def then paper name = "abc" + " & " + "def" + date
- if more than two author, such as abc, def,ghi, then paper name = "abc " + " et al. " + date
- if author name is already of form "abc et al" then paper name = author name


In [26]:


def getPaperName(authors, year):
    if "et al" in authors:
        return authors + " " + year
    
    nameList = authors.split(" and ")

    if len(nameList) == 1:
        return nameList[0].split(",")[0] + " " + year
    elif len(nameList) == 2:
        return nameList[0].split(",")[0] + " & " + nameList[1].split(",")[0] + " " + year
    elif len(nameList) > 2:
        return nameList[0].split(",")[0] + " et al. " + year
    return ""


data["Citation Key"] = data.apply(lambda row: getPaperName(row["Authors"],str(row["Year"])), axis=1 )



## Link to the bibtex file


In [27]:

import json

f = open('ZoteroFile.json')
 
zoteroFile = json.load(f)
zoteroDF = pd.DataFrame(zoteroFile["items"])



In [28]:
"""
#key below is from data and values is a key in zoteroDF
zoteroExcellMap = {key: None for key in data.index}


#make dict based on year matches
data_unique = data.drop_duplicates(subset="Year").dropna(subset="Year")

zoteroDF_unique = zoteroDF.copy()
zoteroDF_unique["Year"] = pd.to_datetime(zoteroDF_unique["date"],errors='coerce').dt.year
zoteroDF_unique = zoteroDF_unique.drop_duplicates(subset="Year").dropna(subset="Year")
dictFromTitleComparison = dict(data_unique.merge(zoteroDF_unique,on="Year",how="left")[["key"]].dropna())


#make a dict mapping based on abstract matches
data_unique = data.drop_duplicates(subset="Abstract").dropna(subset="Abstract")
zoteroDF_unique = zoteroDF.drop_duplicates(subset="abstractNote").dropna(subset="abstractNote")
merged_df = data_unique.merge(zoteroDF_unique, left_on="Abstract", right_on="abstractNote", how="left")
dictFromAbstractComparison = pd.Series(merged_df["key"].values, index=merged_df.index).dropna().to_dict()


#make a dict mapping based on title matches
data_unique = data.drop_duplicates(subset="Title").dropna(subset="Title")
zoteroDF_unique = zoteroDF.drop_duplicates(subset="title").dropna(subset="title")
merged_df = data_unique.merge(zoteroDF_unique,left_on="Title",right_on="title",how="left")
dictFromTitleComparison = pd.Series(merged_df["key"].values, index=merged_df.index).dropna().to_dict()
"""

'\n#key below is from data and values is a key in zoteroDF\nzoteroExcellMap = {key: None for key in data.index}\n\n\n#make dict based on year matches\ndata_unique = data.drop_duplicates(subset="Year").dropna(subset="Year")\n\nzoteroDF_unique = zoteroDF.copy()\nzoteroDF_unique["Year"] = pd.to_datetime(zoteroDF_unique["date"],errors=\'coerce\').dt.year\nzoteroDF_unique = zoteroDF_unique.drop_duplicates(subset="Year").dropna(subset="Year")\ndictFromTitleComparison = dict(data_unique.merge(zoteroDF_unique,on="Year",how="left")[["key"]].dropna())\n\n\n#make a dict mapping based on abstract matches\ndata_unique = data.drop_duplicates(subset="Abstract").dropna(subset="Abstract")\nzoteroDF_unique = zoteroDF.drop_duplicates(subset="abstractNote").dropna(subset="abstractNote")\nmerged_df = data_unique.merge(zoteroDF_unique, left_on="Abstract", right_on="abstractNote", how="left")\ndictFromAbstractComparison = pd.Series(merged_df["key"].values, index=merged_df.index).dropna().to_dict()\n\n\n#make

In [29]:
"""
#update zoteroExcellMap using abtract matches
zoteroExcellMap.update(dictFromAbstractComparison)
print("Found matches for ", len(dictFromAbstractComparison), " rows  and unmatched rows number is ", len(zoteroExcellMap)-len(dictFromAbstractComparison))
"""
"""
#try matching with titles
for k in dictFromTitleComparison.keys():
    if zoteroExcellMap[k] == None:
        zoteroExcellMap[k] = dictFromTitleComparison[k]
    elif  zoteroExcellMap[k] != None and zoteroExcellMap[k] != dictFromTitleComparison[k]:
        print("disagreement with ", k, zoteroExcellMap[k], dictFromTitleComparison[k])
"""

'\n#try matching with titles\nfor k in dictFromTitleComparison.keys():\n    if zoteroExcellMap[k] == None:\n        zoteroExcellMap[k] = dictFromTitleComparison[k]\n    elif  zoteroExcellMap[k] != None and zoteroExcellMap[k] != dictFromTitleComparison[k]:\n        print("disagreement with ", k, zoteroExcellMap[k], dictFromTitleComparison[k])\n'

In [30]:
data_unique = data.drop_duplicates(subset="Abstract").dropna(subset="Abstract")
zoteroDF_unique = zoteroDF.drop_duplicates(subset="abstractNote").dropna(subset="abstractNote")

mergedDF = data_unique.merge(zoteroDF_unique,left_on="Abstract",right_on="abstractNote",how="left")[["Citation Key",'Authors', 'Year', 'Journal', 'Journal ISO Abbreviation', 'Title',
       'Abstract', 'Keywords', 'Open Access', 'Reviewer 1', 'Reviewer 2',
       'Scope', 'Eco-Evo Focus', 'Life history', 'Ecological Loci/Traits',
       'Additional Loci/Traits', 'Mating system', 'Ploidy', 'Selection',
       'Spatial Structure ', 'Population Size', 'Ecological Model',
       'Recurrent Mutation', 'IBS',"DOI","ISSN","url"]]
mergedDF.to_excel("./dataZoteroMerged.xlsx")
#len(mergedDF["abstractNote"].unique())/len(mergedDF["Abstract_x"].unique())
#print(len(data.drop_duplicates(subset="Abstract").dropna(subset="Abstract")["Abstract"]),len(data.index))

In [31]:
len( data_unique.merge(zoteroDF_unique,left_on="Abstract",right_on="abstractNote",how="left")["abstractNote"].unique())

216

## Export processed datafile


In [32]:
exportingDF = data[["Citation Key",'Authors', 'Year', 'Journal', 'Journal ISO Abbreviation', 'Title',
       'Abstract', 'Keywords', 'Open Access', 'Reviewer 1', 'Reviewer 2',
       'Scope', 'Eco-Evo Focus', 'Life history', 'Ecological Loci/Traits',
       'Additional Loci/Traits', 'Mating system', 'Ploidy', 'Selection',
       'Spatial Structure ', 'Population Size', 'Ecological Model',
       'Recurrent Mutation', 'IBS']]
#exportingDF.to_excel("./../data.xlsx")
